In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master("local") \
    .appName("XZ") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
df = spark\
        .read\
        .format("csv")\
        .option("header", "true")\
        .option("inferSchema", "true")\
        .load("/FileStore/tables/retail-data/by-day/*.csv")
    
df.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: timestamp (nullable = true)
-- UnitPrice: double (nullable = true)
-- CustomerID: double (nullable = true)
-- Country: string (nullable = true)

In [3]:
df.rdd.getNumPartitions()

Out[3]: 10

In [4]:
df = df.coalesce(5)
df.rdd.getNumPartitions()

Out[4]: 5

In [5]:
df.cache()
df.createOrReplaceTempView("dfTable")

In [6]:
df.show(2, False)

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode|Description |Quantity|InvoiceDate |UnitPrice|CustomerID|Country |
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
580538 |23084 |RABBIT NIGHT LIGHT |48 |2011-12-05 08:38:00|1.79 |14075.0 |United Kingdom|
580538 |23077 |DOUGHNUT LIP GLOSS |20 |2011-12-05 08:38:00|1.25 |14075.0 |United Kingdom|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows

In [7]:
# this is Action
df.count()

Out[7]: 541909

In [8]:
# this is Transformation
df.select(count("*")).show()

+--------+
count(1)|
+--------+
 541909|
+--------+

### WARNING
There are a number of gotchas when it comes to null values and counting. For instance, when
performing a count(*), Spark will count null values (including rows containing all nulls). However,
when counting an individual column, Spark will not count the null values.

In [10]:
df.select(countDistinct("StockCode")).show()

+-------------------------+
count(DISTINCT StockCode)|
+-------------------------+
 4070|
+-------------------------+

In [11]:
# appriximation is much faster, especially on larger data sets
df.select(approxCountDistinct("StockCode", 0.1)).show()

+--------------------------------+
approx_count_distinct(StockCode)|
+--------------------------------+
 3364|
+--------------------------------+

In [12]:
df.select(first("StockCode"), last("StockCode")).show()

+-----------------------+----------------------+
first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
 23084| 22168|
+-----------------------+----------------------+

In [13]:
df.select(min("UnitPrice"), max("UnitPrice")).show()

+--------------+--------------+
min(UnitPrice)|max(UnitPrice)|
+--------------+--------------+
 -11062.06| 38970.0|
+--------------+--------------+

In [14]:
df.select(sum("Quantity"), sumDistinct("Quantity")).show()

+-------------+----------------------+
sum(Quantity)|sum(DISTINCT Quantity)|
+-------------+----------------------+
 5176450| 29310|
+-------------+----------------------+

In [15]:
df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))\
  .show()

+------------------+---------------+----------------+----------------+
total_transactions|total_purchases| avg_purchases| mean_purchases|
+------------------+---------------+----------------+----------------+
 541909| 5176450|9.55224954743324|9.55224954743324|
+------------------+---------------+----------------+----------------+

In [16]:
df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))\
  .selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases")\
  .show()

+--------------------------------------+----------------+----------------+
(total_purchases / total_transactions)| avg_purchases| mean_purchases|
+--------------------------------------+----------------+----------------+
 9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+

In [17]:
df.select(
    var_pop("Quantity"), 
    var_samp("Quantity"),
    stddev_pop("Quantity"),
    stddev_samp("Quantity")).show()

+-----------------+------------------+--------------------+---------------------+
var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
47559.30364660885| 47559.39140929855| 218.0809566344775| 218.0811578502337|
+-----------------+------------------+--------------------+---------------------+

In [18]:
df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+-------------------+------------------+
 skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
-0.2640755761052859|119768.05495534562|
+-------------------+------------------+

In [19]:
df.select(
    corr("UnitPrice", "Quantity"), 
    covar_samp("InvoiceNo", "Quantity"),
    covar_pop("InvoiceNo", "Quantity"))\
.show(1, False)

+-------------------------+-------------------------------+------------------------------+
corr(UnitPrice, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
-0.0012349245448703078 |1052.728054386325 |1052.726077870221 |
+-------------------------+-------------------------------+------------------------------+

In [20]:
df.select(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+

In [21]:
df.groupBy("InvoiceNo").count().orderBy(col("count").desc()).show(2)

+---------+-----+
InvoiceNo|count|
+---------+-----+
 573585| 1114|
 581219| 749|
+---------+-----+
only showing top 2 rows

In [22]:
df\
  .groupBy("InvoiceNo")\
  .agg(
      count("Quantity").alias("quan"),
      expr("count(Quantity)"))\
  .show(2)

+---------+----+---------------+
InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
 574966| 8| 8|
 575091| 38| 38|
+---------+----+---------------+
only showing top 2 rows

In [23]:
df\
  .groupBy("InvoiceNo")\
  .agg(
      {"Quantity": "count"})\
  .show(2)

+---------+---------------+
InvoiceNo|count(Quantity)|
+---------+---------------+
 574966| 8|
 575091| 38|
+---------+---------------+
only showing top 2 rows

### Window Functions

In [25]:
xz = spark\
      .range(9)\
      .withColumn("bucket", col("id") % 3)\
      
xz.show()
  

+---+------+
 id|bucket|
+---+------+
 0| 0|
 1| 1|
 2| 2|
 3| 0|
 4| 1|
 5| 2|
 6| 0|
 7| 1|
 8| 2|
+---+------+

In [26]:
from pyspark.sql.window import Window

wnd = Window.partitionBy("bucket")

xz\
  .withColumn("max_over_partitioon", max("id").over(wnd))\
  .show()

+---+------+-------------------+
 id|bucket|max_over_partitioon|
+---+------+-------------------+
 0| 0| 6|
 3| 0| 6|
 6| 0| 6|
 1| 1| 7|
 4| 1| 7|
 7| 1| 7|
 2| 2| 8|
 5| 2| 8|
 8| 2| 8|
+---+------+-------------------+

In [27]:
wnd = Window.partitionBy("bucket").rowsBetween(Window.unboundedPreceding, Window.currentRow)

xz\
  .withColumn("max_over_partitioon", max("id").over(wnd))\
  .show()

+---+------+-------------------+
 id|bucket|max_over_partitioon|
+---+------+-------------------+
 0| 0| 0|
 3| 0| 3|
 6| 0| 6|
 1| 1| 1|
 4| 1| 4|
 7| 1| 7|
 2| 2| 2|
 5| 2| 5|
 8| 2| 8|
+---+------+-------------------+

In [28]:
from pyspark.sql.window import Window

wnd = Window.partitionBy("bucket").orderBy("id")

xz\
  .withColumn("rank", rank().over(wnd))\
  .show()

+---+------+----+
 id|bucket|rank|
+---+------+----+
 0| 0| 1|
 3| 0| 2|
 6| 0| 3|
 1| 1| 1|
 4| 1| 2|
 7| 1| 3|
 2| 2| 1|
 5| 2| 2|
 8| 2| 3|
+---+------+----+

In [29]:
# https://www.youtube.com/watch?v=hoOwDUrodhM
from pyspark.sql.window import Window

wnd = Window.partitionBy("bucket").orderBy("id")

xz\
  .withColumn("dense rank", dense_rank().over(wnd))\
  .show()

+---+------+----------+
 id|bucket|dense rank|
+---+------+----------+
 0| 0| 1|
 3| 0| 2|
 6| 0| 3|
 1| 1| 1|
 4| 1| 2|
 7| 1| 3|
 2| 2| 1|
 5| 2| 2|
 8| 2| 3|
+---+------+----------+

In [30]:
# https://www.youtube.com/watch?v=hoOwDUrodhM
from pyspark.sql.window import Window

wnd = Window.partitionBy("bucket").orderBy("id")

xz\
  .withColumn("row number", row_number().over(wnd))\
  .show()

+---+------+----------+
 id|bucket|row number|
+---+------+----------+
 0| 0| 1|
 3| 0| 2|
 6| 0| 3|
 1| 1| 1|
 4| 1| 2|
 7| 1| 3|
 2| 2| 1|
 5| 2| 2|
 8| 2| 3|
+---+------+----------+

In [31]:
from pyspark.sql.window import Window

wnd = Window.partitionBy("bucket").orderBy("id")

xz\
  .withColumn("lead", lead("id").over(wnd))\
  .show()

+---+------+----+
 id|bucket|lead|
+---+------+----+
 0| 0| 3|
 3| 0| 6|
 6| 0|null|
 1| 1| 4|
 4| 1| 7|
 7| 1|null|
 2| 2| 5|
 5| 2| 8|
 8| 2|null|
+---+------+----+

In [32]:
from pyspark.sql.window import Window

wnd = Window.partitionBy("bucket").orderBy("id")

xz\
  .withColumn("lag", lag("id", 1).over(wnd))\
  .show()

+---+------+----+
 id|bucket| lag|
+---+------+----+
 0| 0|null|
 3| 0| 0|
 6| 0| 3|
 1| 1|null|
 4| 1| 1|
 7| 1| 4|
 2| 2|null|
 5| 2| 2|
 8| 2| 5|
+---+------+----+

In [33]:
from pyspark.sql.window import Window

wnd = Window\
        .partitionBy("bucket")\
        .orderBy("id")
        # looks like this is the default for sum. for max/min, the default is the whole partition/frame/window
        #.rowsBetween(Window.unboundedPreceding, Window.currentRow)

xz\
  .withColumn("runing id sum", sum("id").over(wnd))\
  .show()

+---+------+-------------+
 id|bucket|runing id sum|
+---+------+-------------+
 0| 0| 0|
 3| 0| 3|
 6| 0| 9|
 1| 1| 1|
 4| 1| 5|
 7| 1| 12|
 2| 2| 2|
 5| 2| 7|
 8| 2| 15|
+---+------+-------------+

In [34]:
# Rollups

dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate\
    .where("CustomerID = 12347")\
    .select("CustomerID", "*")\
    .show(20, False)

windowSpec = Window\
  .partitionBy("CustomerId", "date")\
  .orderBy(asc("Quantity"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

dfWithDate.where("CustomerId = 12347").orderBy("CustomerId")\
  .select(
      col("CustomerId"),
      col("date"),
      col("Quantity"),
      purchaseRank.alias("quantityRank"),
      purchaseDenseRank.alias("quantityDenseRank"),
      maxPurchaseQuantity.alias("maxPurchaseQuantity"))\
.show()


+----------+---------+---------+-----------------------------------+--------+-------------------+---------+----------+-------+----------+
CustomerID|InvoiceNo|StockCode|Description |Quantity|InvoiceDate |UnitPrice|CustomerID|Country|date |
+----------+---------+---------+-----------------------------------+--------+-------------------+---------+----------+-------+----------+
12347.0 |573511 |23480 |MINI LIGHTS WOODLAND MUSHROOMS |12 |2011-10-31 12:25:00|3.75 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |21265 |PINK GOOSE FEATHER TREE 60CM |12 |2011-10-31 12:25:00|1.95 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |21636 |MADRAS NOTEBOOK MEDIUM |12 |2011-10-31 12:25:00|0.75 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |22372 |AIRLINE BAG VINTAGE WORLD CHAMPION |4 |2011-10-31 12:25:00|4.25 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |22375 |AIRLINE BAG VINTAGE JET SET BROWN |8 |2011-10-31 12:25:00|4.25 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |22371 |AIRLINE BAG VINTAGE TOKYO 78 |12 |2011-10-31 12:25:00|3.95 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |22374 |AIRLINE BAG VINTAGE JET SET RED |8 |2011-10-31 12:25:00|4.25 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |22252 |BIRDCAGE DECORATION TEALIGHT HOLDER|12 |2011-10-31 12:25:00|1.25 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |22945 |CHRISTMAS METAL TAGS ASSORTED |12 |2011-10-31 12:25:00|0.85 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |22423 |REGENCY CAKESTAND 3 TIER |6 |2011-10-31 12:25:00|12.75 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |23173 |REGENCY TEAPOT ROSES |2 |2011-10-31 12:25:00|9.95 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |47580 |TEA TIME DES TEA COSY |6 |2011-10-31 12:25:00|2.55 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |47567B |TEA TIME KITCHEN APRON |6 |2011-10-31 12:25:00|5.95 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |47559B |TEA TIME OVEN GLOVE |10 |2011-10-31 12:25:00|1.25 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |22698 |PINK REGENCY TEACUP AND SAUCER |12 |2011-10-31 12:25:00|2.95 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |22697 |GREEN REGENCY TEACUP AND SAUCER |12 |2011-10-31 12:25:00|2.95 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |84558A |3D DOG PICTURE PLAYING CARDS |36 |2011-10-31 12:25:00|2.95 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |23084 |RABBIT NIGHT LIGHT |48 |2011-10-31 12:25:00|1.79 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |21731 |RED TOADSTOOL LED NIGHT LIGHT |12 |2011-10-31 12:25:00|1.65 |12347.0 |Iceland|2011-10-31|
12347.0 |573511 |23177 |TREASURE ISLAND BOOK BOX |8 |2011-10-31 12:25:00|2.25 |12347.0 |Iceland|2011-10-31|
+----------+---------+---------+-----------------------------------+--------+-------------------+---------+----------+-------+----------+
only showing top 20 rows

+----------+----------+--------+------------+-----------------+-------------------+
CustomerId| date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
 12347.0|2010-12-07| 3| 1| 1| 3|
 12347.0|2010-12-07| 4| 2| 2| 4|
 12347.0|2010-12-07| 4| 2| 2| 4|
 12347.0|2010-12-07| 4| 2| 2| 4|
 12347.0|2010-12-07| 4| 2| 2| 4|
 12347.0|2010-12-07| 4| 2| 2| 4|
 12347.0|2010-12-07| 4| 2| 2| 4|
 12347.0|2010-12-07| 4| 2| 2| 4|
 12347.0|2010-12-07| 6| 9| 3| 6|
 12347.0|2010-12-07| 6| 9| 3| 6|
 12347.0|2010-12-07| 6| 9| 3| 6|
 12347.0|2010-12-07| 6| 9| 3| 6|
 12347.0|2010-12-07| 6| 9| 3| 6|
 12347.0|2010-12-07| 6| 9| 3| 6|
 12347.0|2010-12-07| 6| 9| 3| 6|
 12347.0|2010-12-07| 12| 16| 4| 12|
 12347.0|2010-12-07| 12| 16| 4| 12|
 12347.0|2010-12-07| 12| 16| 4| 12|
 12347.0|2010-12-07| 12| 16| 4| 12|
 12347.0|2010-12-07| 12| 16| 4| 12|
+----------+----------+--------+------------+-----------------+-------------------+
only showing top 20 rows

In [35]:
dfNoNull = dfWithDate.drop()

rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date", "Country")
rolledUpDF.show()

+----------+--------------+--------------+
 Date| Country|total_quantity|
+----------+--------------+--------------+
 null| null| 5176450|
2010-12-01| null| 26814|
2010-12-01| Australia| 107|
2010-12-01| EIRE| 243|
2010-12-01| France| 449|
2010-12-01| Germany| 117|
2010-12-01| Netherlands| 97|
2010-12-01| Norway| 1852|
2010-12-01|United Kingdom| 23949|
2010-12-02| null| 21023|
2010-12-02| EIRE| 4|
2010-12-02| Germany| 146|
2010-12-02|United Kingdom| 20873|
2010-12-03| null| 14830|
2010-12-03| Belgium| 528|
2010-12-03| EIRE| 2575|
2010-12-03| France| 239|
2010-12-03| Germany| 170|
2010-12-03| Italy| 164|
2010-12-03| Poland| 140|
+----------+--------------+--------------+
only showing top 20 rows

In [36]:
dfNoNull = dfWithDate.drop()

rolledUpDF = dfNoNull.rollup("Date", "Country", "StockCode").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "StockCode", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date", "Country")
rolledUpDF.show()

+----------+---------+---------+--------------+
 Date| Country|StockCode|total_quantity|
+----------+---------+---------+--------------+
 null| null| null| 5176450|
2010-12-01| null| null| 26814|
2010-12-01|Australia| 22193| 2|
2010-12-01|Australia| 22726| 4|
2010-12-01|Australia| 22192| 2|
2010-12-01|Australia| 22191| 2|
2010-12-01|Australia| 21622| 8|
2010-12-01|Australia| 85014B| 6|
2010-12-01|Australia| null| 107|
2010-12-01|Australia| 85014A| 3|
2010-12-01|Australia| 22941| 6|
2010-12-01|Australia| 22727| 4|
2010-12-01|Australia| 21791| 12|
2010-12-01|Australia| 35004C| 6|
2010-12-01|Australia| 35004G| 4|
2010-12-01|Australia| 22196| 24|
2010-12-01|Australia| 22195| 24|
2010-12-01| EIRE| 22968| 4|
2010-12-01| EIRE| 21056| 4|
2010-12-01| EIRE| 21055| 4|
+----------+---------+---------+--------------+
only showing top 20 rows

In [37]:
dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
.select("Date", "Country", "sum(Quantity)").orderBy("Date").show()

+----+--------------------+-------------+
Date| Country|sum(Quantity)|
+----+--------------------+-------------+
null| Japan| 25218|
null| Portugal| 16180|
null| Australia| 83653|
null| RSA| 352|
null| Cyprus| 6317|
null| Germany| 117448|
null| Lebanon| 386|
null| Channel Islands| 9479|
null| Unspecified| 3300|
null| null| 5176450|
null| Finland| 10666|
null|United Arab Emirates| 982|
null| Hong Kong| 4769|
null| Spain| 26824|
null| USA| 1034|
null| Czech Republic| 592|
null| European Community| 497|
null| Norway| 19247|
null| Singapore| 5234|
null| Denmark| 8188|
+----+--------------------+-------------+
only showing top 20 rows